# Análisis final

Equipo 2

Modelación estadística para la toma de decisiones

## Librerías básicas

In [1]:
import numpy as np
import pandas as pd
import basic_stats as bs

## Carga de la base de datos

In [2]:
df = pd.read_pickle('clean_model_sample.pkl')
df.shape

(94892, 7)

In [3]:
df.head()

,INTUBADO,SEXO,EDAD,TABAQUISMO,OBESIDAD,ASMA,DIABETES
2298933,0,1,0.83,0,0,0,0
6251919,0,0,0.43,0,0,0,0
3910108,0,0,0.51,0,0,0,0
3203275,0,0,0.75,0,0,0,0
224077,0,0,0.75,0,0,0,1


## Proporciones de intubados por sexo

### Proporción general

In [4]:
total = len(df)
intub = df[df['INTUBADO'] == 1]
intub_total = len(intub)
intub_perc = intub_total / total
print(f'p = {intub_perc:.5f}')

p = 0.10411


#### Prueba

Dar un intervalo de confianza

In [5]:
alpha = 0.05
n = intub_total
z = - bs.normal_ppf(alpha, 'center')
change = z * np.sqrt(intub_perc * (1 - intub_perc) / n)
print(f'CI = [{intub_perc - change:.5f}, {intub_perc + change:.5f}] al {100 * (1 - alpha)}%')

CI = [0.09809, 0.11013] al 95.0%


Este es el intervalo de confianza para la proporción poblacional de mexicanos intubados

### Proporción de hombres y mujeres

In [6]:
intub_male = intub[intub['SEXO'] == 0]
intub_female = intub[intub['SEXO'] == 1]
prop_male = len(intub_male) / intub_total
prop_female = 1 - prop_male
print(f'''
p_m = {prop_male:.4f}
p_f = {prop_female:.4f}
''')


p_m = 0.6237
p_f = 0.3763



### Prueba de hipótesis

¿Qué tanta diferencia hay entre las proporciones de hombres y mujeres intubados? Nuestra hipótesis es que los hombres llegan a intubarse más seguido que las mujeres

$$
H_{o}: p_h - p_m = \delta
\\
H_{a}: p_h - p_m \geq \delta
$$

In [7]:
delta = 0.22
# Probar para diferentes valores de delta, la hipótesis nula es que la diferencia entre las proporciones (hombre - mujer) = delta
n_male = len(intub_male)
n_female = len(intub_female)
z_alt = (prop_male - prop_female - delta) / np.sqrt(intub_perc * (1 - intub_perc) * (1 / n_male + 1 / n_female))
z_null = bs.normal_ppf(alpha, 'right')
p_value = 1 - bs.normal_cdf(z_alt, 0, 1)
print(f'''z_alt = {z_alt}
z_null = {z_null}''')
print(f'''p_value = {p_value}
p_value < alpha ?= {p_value < alpha}''')

z_alt = 4.3349074590825944
z_null = 1.6448536269514722
p_value = 7.291072676940402e-06
p_value < alpha ?= True


Hay evidencia muy fuerte para afirmar que la diferencia de las proporciones de hombres y mujeres intubados es mayor del 22%, es decir, hay evidencia estadística suficiente para afirmar que los hombres son más susceptibles a ser intubados

## Modelo de regresión logística

Se busca generar un modelo de regresión logística que pueda clasificar a cada un individuo como intubado o no intubado en base a la presencia de diferentes morbilidades, su edad y su sexo.

### Librerías básicas

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

In [9]:
df2 = pd.read_pickle('clean_model_sample.pkl')

### Creación de datos de prueba

In [10]:
variables = ['SEXO', 'EDAD', 'DIABETES', 'TABAQUISMO', 'OBESIDAD', 'ASMA']
X_train, X_test, y_train, y_test = train_test_split(df2[variables], 
                                    df2.INTUBADO, test_size=0.3, random_state=42)

### Entrenamiento del modelo

In [11]:
model = LogisticRegression(max_iter=1000, class_weight='auto')
model.fit(X_train, y_train)

LogisticRegression(class_weight='auto', max_iter=1000)

### Predicciones de la categorización de los individuos en los datos de prueba

In [12]:
predictions = model.predict(X_test)
np.count_nonzero(predictions == 1)

0

In [13]:
np.count_nonzero(y_test == 1)

2985

### Probabilidad de resultados

Se muestra en un formato tabular la probabilidad de que cada individuo (renglón) sea categorizado como intubado o no intubado (columna)

In [14]:
model.predict_proba(X_test)

array([[0.91100174, 0.08899826],
       [0.91440944, 0.08559056],
       [0.93786038, 0.06213962],
       ...,
       [0.87287556, 0.12712444],
       [0.93372867, 0.06627133],
       [0.82391864, 0.17608136]])

### Pesos de cada variable

In [15]:
coef = model.coef_[0]
coef = [np.exp(x)/(1 + np.exp(x)) for x in coef]
for i, v in enumerate(variables):
    print(f'{v:<10} {coef[i]:.5f}')

SEXO       0.42115
EDAD       0.68971
DIABETES   0.52555
TABAQUISMO 0.48082
OBESIDAD   0.60617
ASMA       0.47138


### Métricas del modelo

In [16]:
accuracy = accuracy_score(y_test, predictions)
roc = roc_auc_score(y_test, predictions)
print(f'''
Precisión = {accuracy * 100:.6f}%
ROC_AUC = {roc * 100:.6f}%
''')


Precisión = 89.514543%
ROC_AUC = 50.000000%



In [17]:
test_subject = np.array([[0, 0.5, 0, 0, 0, 0]])
model.predict_proba(test_subject)

array([[0.8980371, 0.1019629]])